# Build a Microsoft Authenticator-like Time-based OTP Algorithm

In [7]:
# Required Python modules

import time   # Generates time-based counters for TOTP
import hmac   # Creates secure HMACs for OTP Generation
import hashlib  # Provides SHA-1 Hashing for HMAC
import base64   # Encodes and decodes the secret key
import struct   # Converts hashed bytes into numeric OTPs

### Class containing the methods for TOTP Authenticator

In [8]:
class TOTPAuthenticator:

    # Constructor & Key Storage - encoding the secret key using Base32 (shared during initial setup)
    def __init__(self, secret_key, interval=30, digits=6):
        self.secret_key = secret_key  # Base32-encoded key shared between the server and client for code generation and validation
        self.interval = interval  # Time window (default = 30 seconds) for each code’s validity
        self.digits = digits # Specifies the length of the generated OTP (default = 6 digits)
        self.generated_codes = []  # Using array to store recent codes (simple queue)

    # Hash Generation
    def _generate_hmac(self, counter):
        key = base64.b32decode(self.secret_key, True)   # Decode the secret key from Base32 to bytes
        msg = struct.pack(">Q", counter)  # Convert the counter to bytes
        hmac_hash = hmac.new(key, msg, hashlib.sha1).digest()   # Generate HMAC-SHA1 hash
        return hmac_hash

    # HMAC Generation, Dynamic Truncation, Circular Queue
    def generate_code(self):
        counter = int(time.time() // self.interval)   # Generate counter based on current time
        hmac_hash = self._generate_hmac(counter)

        # Dynamic Truncation (select 4 bytes)
        offset = hmac_hash[-1] & 0x0F
        truncated_hash = hmac_hash[offset:offset + 4]
        code = struct.unpack(">I", truncated_hash)[0] & 0x7FFFFFFF

        # Extract the desired number of digits
        otp = str(code % (10 ** self.digits)).zfill(self.digits)

        # Append to array (acts as circular queue for recent codes)
        if len(self.generated_codes) >= 5:  # Store last 5 codes
            self.generated_codes.pop(0)
        self.generated_codes.append(otp)

        return otp

    # Code Validation
    def validate_code(self, input_code):
        # Allow slight time drift using recent codes
        return input_code in self.generated_codes

### Test the TOTP Authenticator

In [9]:
# Shared between the server and client during initial setup

secret_key = 'JBSWY3DPEHPK3PXP'

In [10]:
# Initialize Authenticator

authenticator = TOTPAuthenticator(secret_key)

In [11]:
# Generate a code

code = authenticator.generate_code()
print(f"Generated Code: {code}")

Generated Code: 242790


In [12]:
# Validate the code

user_input = input("Enter the code: ")

if authenticator.validate_code(user_input):
  print("Code is valid!")
else:
  print("Invalid code!")

Enter the code: 242790
Code is valid!
